## Прогнозирование стоимости автомобиля по характеристикам


In [ ]:
# %pip install catboost
# %pip install sklearn
# %pip install tqdm
# %pip install phik
# %pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
# %pip install seaborn
# %pip install pandas-profiling[notebook]

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
# %pip freeze > requirements.txt
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))
VERSION    = 2
DIR_TRAIN  = './input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = './input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%    

Python       : 3.10.1 (tags/v3.10.1:2cd268a, Dec  6 2021, 19:10:37) [MSC v.1929 64 bit (AMD64)]
Numpy        : 1.22.0


# Data

In [2]:

# train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')
train_my = pd.read_csv('dftrain.csv') # мой датасет для обучения модели



In [ ]:
# %pip uninstall pandas-profiling
# %pip install pandas-profiling[notebook,html]
# %pip install ruamel-yaml

#использование профилировщика пандас для изучения тестового датасета
# from pandas_profiling import ProfileReport
# profile = ProfileReport(test, title="Pandas Profiling Report")
# profile.to_file("your_report.html")
# # train[['bodyType','brand','color','fuelType','modelDate','mileage']].head(3)
# train.sample(3)
# test[['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
#        'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
#        'fuelType', 'image', 'mileage']]
# # train[['bodyType','brand','color','fuelType','modelDate','mileage']].head(3)
# train[['bodyType', 'brand', 'productionDate', 'engineDisplacement', 'mileage']].sample(3)
# inters=set(train.columns).intersection(test.columns)
# inters=list(inters)
# inters.sort()
# print(inters)
# dif=list(set(test.columns).difference(inters))
# dif.sort()
# print(dif)
# train.info()
# test['brand'].value_counts()
# train['brand'].value_counts()
# modelDate=test.modelDate.value_counts()
# modelDate.loc[modelDate>2]
# productionDate=test.productionDate.value_counts()
# productionDate.loc[productionDate>1]

## Data Preprocessing

In [17]:
# train.dropna(subset=['productionDate','mileage'], inplace=True)
# train.dropna(subset=['price'], inplace=True)
# # для baseline просто возьму пару схожих признаков без полной обработки
# columns = ['bodyType', 'brand', 'productionDate', 'engineDisplacement', 'mileage']

# df_train = train[columns]
# df_test = test[columns]
# y = train['price']

In [11]:
#из-за нехватки времени некоторые признаки в тренировочной базе являются неполными
# адаптировать тестовый датасет, чтобы там не было отсутствующих значений для  таких признаков:
mycolors=['чёрный', 'белый', 'серый', 'синий', 'серебристый','красный']
test['color']=test['color'].apply(lambda x: x if (x in mycolors) else 'другой цвет')
test['fuelType']=test['fuelType'].apply(lambda x: x if (x in ['бензин','дизель','гибрид']) else 'другое топливо')


чёрный         11010
белый           6133
другой цвет     4626
серебристый     4113
серый           3984
синий           3289
красный         1531
Name: color, dtype: int64

In [ ]:
# 'complectation_dict','equipment_dict','model_info','super_gen',
columns_my=['bodyType', 'brand', 'color', 
        'engineDisplacement', 'enginePower', 
       'fuelType',  'mileage', 'modelDate',  'model_name',
       'name', 'numberOfDoors',  
       'productionDate', 'sell_id',  'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы',  'ПТС',
       'Привод', 'Руль']
columns_my = ['bodyType', 'brand', 'productionDate', 'engineDisplacement', 'mileage',
'enginePower',         'modelDate'
]
df_train_my = train_my[columns_my]
df_test = test[columns_my]
y = train_my['price']

In [3]:
df_train_my['fuelType'].value_counts()

NameError: name 'df_train_my' is not defined

In [36]:
len(df_train_my['enginePower'].value_counts().values)

330

In [ ]:
df_train_my['numberOfDoors']=df_train_my['numberOfDoors'].apply(lambda x: int(x))
df_train_my.sample(3).iloc[:,5:]

## Label Encoding

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
# df_train['sample'] = 1 # помечаем где у нас трейн
# df_test['sample'] = 0 # помечаем где у нас тест

# data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [5]:
df_train_my['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
data_my = df_test.append(df_train_my, sort=False).reset_index(drop=True) # объединяем
data_my.sample(3).iloc[:,:]

C:\Users\User\AppData\Local\Temp/ipykernel_8416/501939137.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_my['sample'] = 1 # помечаем где у нас трейн
C:\Users\User\AppData\Local\Temp/ipykernel_8416/501939137.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['sample'] = 0 # помечаем где у нас тест


,bodyType,brand,productionDate,engineDisplacement,mileage,enginePower,modelDate,sample
2219,седан,SKODA,2005.0,2.5 LTR,309000.0,163 N12,2001.0,0
13905,внедорожник 5 дв.,NISSAN,2007.0,2.2 LTR,280000.0,136 N12,2000.0,0
27811,внедорожник 5 дв.,MITSUBISHI,2011.0,2.0 LTR,123600.0,147 N12,2009.0,0


In [ ]:
data_my['color'].value_counts()
#скорректировать цвета в исходном датасете (убрать цвета, которых не получилось у меня собрать)

In [44]:
# для катбуста не нужно собственноручно переводить в категорию
# for colum in ['bodyType', 'brand', 'engineDisplacement']:
#     data[colum] = data[colum].astype('category').cat.codes
# X = data.query('sample == 1').drop(['sample'], axis=1)
# X_sub = data.query('sample == 0').drop(['sample'], axis=1)


In [6]:
# , 'complectation_dict'
# , 'equipment_dict',
# 'mileage',
#  'model_info',
# 
# 'parsing_unixtime',
# 'sell_id',
# 'super_gen','modelDate','productionDate', 
cat_features_ids=['bodyType', 'brand', 'color',
        'engineDisplacement', 'enginePower',
       'fuelType',    'model_name','name',
        'numberOfDoors',  
         'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы',  'ПТС',
       'Привод', 'Руль']
cat_features_ids=['bodyType', 'brand', 'engineDisplacement',
'enginePower',       
]

X = data_my.query('sample == 1').drop(['sample'], axis=1)
X_sub = data_my.query('sample == 0').drop(['sample'], axis=1)


In [43]:
y

0         220000.0
1         490000.0
2         180000.0
3         480000.0
4         440000.0
           ...    
25094     590000.0
25095     220000.0
25096    1070000.0
25097     890000.0
25098     800000.0
Name: price, Length: 25099, dtype: float64

In [ ]:
cat_features_ids

## Train Split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [49]:
# tmp_train = X_train.copy()
# tmp_train['price'] = y_train
# # Находим median по экземплярам engineDisplacement в трейне и размечаем тест
# predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())

# #оцениваем точность
# print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

# # Model 2 : CatBoost


## Fit

In [8]:
X_train

,bodyType,brand,productionDate,engineDisplacement,mileage,enginePower,modelDate
49481,седан,NISSAN,2008.0,1.6 LTR,252000.0,110 N12,2004.0
46476,внедорожник 5 дв.,MITSUBISHI,2015.0,2.4 LTR,95000.0,167 N12,2014.0
46223,хэтчбек 5 дв.,MITSUBISHI,2011.0,1.5 LTR,126000.0,95 N12,2002.0
59263,внедорожник 5 дв.,VOLVO,2003.0,2.4 LTR,200000.0,163 N12,2002.0
49617,внедорожник 5 дв.,NISSAN,2011.0,1.6 LTR,101000.0,190 N12,2010.0
...,...,...,...,...,...,...,...
56261,седан,VOLKSWAGEN,2015.0,2.0 LTR,68000.0,115 N12,2014.0
40076,хэтчбек 5 дв.,BMW,2011.0,1.6 LTR,210000.0,115 N12,2007.0
35546,седан,AUDI,2016.0,3.0 LTR,122000.0,333 N12,2014.0
50481,седан,NISSAN,2002.0,3.5 LTR,352000.0,255 N12,1999.0


In [46]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
        #  plot=True
         )

model.save_model('catboost_single_model_baseline.model')

In [11]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Точность модели по метрике MAPE: 23.24%


Вот так просто со старта, даже не трогая сами данные и не подбирая настройки catboosta, получаем модель с уровнем ошибки в 18%!

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [29]:
# попытки разобраться с признаком parse_unixtime
from datetime import datetime
def parseunix(x):
    ts = int(x)
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

df_=test.loc[:,['parsing_unixtime']] 
df_['unixtime']=df_['parsing_unixtime'].apply(parseunix)

df_.sort_values(by='unixtime',ascending=True)

,parsing_unixtime,unixtime
9705,1603116197,2020-10-19
10407,1603118942,2020-10-19
10408,1603118945,2020-10-19
10409,1603118949,2020-10-19
10410,1603118953,2020-10-19
...,...,...
34374,1603682685,2020-10-26
34373,1603682679,2020-10-26
34372,1603682676,2020-10-26
34370,1603682667,2020-10-26


In [31]:
cat_features_ids

['bodyType', 'brand', 'engineDisplacement', 'enginePower', 'fuelType']

In [32]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, np.log(y_train),
         cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
        #  plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

ValueError: 'enginePower' is not in list

In [14]:
predict_test = np.exp(model.predict(X_test))
predict_submission = np.exp(model.predict(X_sub))

In [15]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 19.33%


Как видим точность возросла до 15%, а что будет на ЛБ?

# Submission

In [17]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_myds_v{VERSION}.csv', index=False)
sample_submission.head(10)
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   sell_id  34686 non-null  int64  
 1   price    34686 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 542.1 KB


В итоге получили **MAPE 27%** на ЛБ!

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing